In [1]:
import numpy as np

In [2]:
import pickle

In [3]:
with open('train_qa.txt','rb') as f:
    train_data = pickle.load(f)

In [4]:
with open('test_qa.txt','rb') as f:
    test_data = pickle.load(f)

In [5]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [6]:

len(test_data)

1000

In [7]:
len(train_data)

10000

# Creating Vocabulary for the entire Data Set

In [8]:
all_data = test_data+train_data

In [9]:
vocab = set()
for story,questions,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(questions))
vocab.add('yes')
vocab.add('no')

In [10]:
len(vocab)

37

In [11]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [12]:
vocab_len = len(vocab) + 1
#we add an extra 1 to hold 0 for Keras' pad_sequences

In [13]:
vocab_len

38

In [14]:
#we also need max story length and max question length for when we train the Model later on
max_story_len = max([len(d[0]) for d in all_data])
max_question_len = min([len(d[1]) for d in all_data])

In [15]:
max_question_len

6

# Writing a function to vectorize the Data

In [16]:
#Creating the vocab size again
vocab_size = len(vocab) + 1

In [17]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [18]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [19]:
tokenizer.word_index

{'apple': 1,
 'to': 2,
 'travelled': 3,
 'yes': 4,
 'milk': 5,
 'kitchen': 6,
 'journeyed': 7,
 'in': 8,
 'got': 9,
 'down': 10,
 'discarded': 11,
 'took': 12,
 'john': 13,
 'put': 14,
 'garden': 15,
 'no': 16,
 'sandra': 17,
 'went': 18,
 'picked': 19,
 'back': 20,
 'hallway': 21,
 'there': 22,
 'bedroom': 23,
 'dropped': 24,
 'moved': 25,
 '.': 26,
 'mary': 27,
 'office': 28,
 'daniel': 29,
 'bathroom': 30,
 'up': 31,
 'football': 32,
 'is': 33,
 'left': 34,
 'grabbed': 35,
 '?': 36,
 'the': 37}

In [20]:
#Creating the function
def vectorize_data(data,word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    
    #The output of the fuction will vectorize whatever data that has been passed in the function
    #The output list will be:-
    STORY = []
    QUESTIONS = []
    ANSWER = []
    
    for s,q,a in data:
        #s = story, q=question, a=answer
        story = [word_index[word.lower()] for word in s]
        question = [word_index[word.lower()] for word in q]
        
        #Since the answer is in YES or NO, it would be easy to use np.arrays to start it of with 0's
        answer = np.zeros(len(word_index) + 1)
        answer[word_index[a]] = 1
        
        #Now we will append everything to the output list
        STORY.append(story)
        QUESTIONS.append(question)
        ANSWER.append(answer)
    
    #finally we will return tuple for unpacking along with pad_sequences
    return(pad_sequences(STORY,maxlen=max_story_len), pad_sequences(QUESTIONS,maxlen=max_question_len), np.array(ANSWER))
    

In [21]:
inputs_train, queries_train, answers_train = vectorize_data(train_data)

In [22]:
inputs_test, queries_test, answers_test = vectorize_data(test_data)

In [23]:
tokenizer.word_index['yes']

4

In [24]:
tokenizer.word_index['no']

16

# CREATING THE MODEL

In [25]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM

Placeholder for Input

In [26]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))
#it is left afte',' because the batch size is not yet chosen at the moment

Encoders

In [27]:
#Input Encoder M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

#Input Encoder C
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len)) #refer the research paper to understand why the output_dim in encoder C is taken as max_question_len
input_encoder_c.add(Dropout(0.3))

#Question Encoder 
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

Instructions for updating:
Colocations handled automatically by placer.


Encode the Sequences

In [28]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

Performing the operation as in the research paper

In [29]:
#dot product
match = dot([input_encoded_m,question_encoded],axes=(2,2))
match = Activation('softmax')(match)

#adding the matrix with the sequence C
response = add([match,input_encoded_c])
response = Permute((2,1))(response)


In [30]:
#final answer
answer = concatenate([response,question_encoded])

In [31]:
answer

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 6, 220) dtype=float32>

In [32]:
#reduce the model with RNN using LSTM
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)

In [33]:
answer = Activation('softmax')(answer)

Building the final model

In [34]:
model = Model([input_sequence,question], answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 156)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
____________________________________________________________________________________________

In [36]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=150,validation_data=([inputs_test, queries_test], answers_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 10000 samples, validate on 1000 samples
Epoch 1/150
10000/10000 [==============================] - 8s 797us/step - loss: 0.9136 - accuracy: 0.5013 - val_loss: 0.6957 - val_accuracy: 0.4970
Epoch 2/150
10000/10000 [==============================] - 7s 699us/step - loss: 0.7047 - accuracy: 0.4958 - val_loss: 0.6936 - val_accuracy: 0.5030
Epoch 3/150
10000/10000 [==============================] - 6s 623us/step - loss: 0.6961 - accuracy: 0.4995 - val_loss: 0.6936 - val_accuracy: 0.4970
Epoch 4/150
10000/10000 [==============================] - 6s 620us/step - loss: 0.6950 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 5/150
10000/10000 [==============================] - 6s 622us/step - loss: 0.6955 - accuracy: 0.4931 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 6/150
10000/10000 [==============================] - 6s 625us/step - loss:

10000/10000 [==============================] - ETA: 0s - loss: 0.3132 - accuracy: 0.86 - 6s 649us/step - loss: 0.3132 - accuracy: 0.8670 - val_loss: 0.3948 - val_accuracy: 0.8260
Epoch 52/150
10000/10000 [==============================] - 6s 643us/step - loss: 0.3144 - accuracy: 0.8658 - val_loss: 0.3639 - val_accuracy: 0.8440
Epoch 53/150
10000/10000 [==============================] - 7s 659us/step - loss: 0.3147 - accuracy: 0.8643 - val_loss: 0.3700 - val_accuracy: 0.8410
Epoch 54/150
10000/10000 [==============================] - 6s 634us/step - loss: 0.3125 - accuracy: 0.8644 - val_loss: 0.3731 - val_accuracy: 0.8430
Epoch 55/150
10000/10000 [==============================] - 6s 633us/step - loss: 0.3123 - accuracy: 0.8657 - val_loss: 0.3690 - val_accuracy: 0.8400
Epoch 56/150
10000/10000 [==============================] - 7s 651us/step - loss: 0.3080 - accuracy: 0.8679 - val_loss: 0.3841 - val_accuracy: 0.8350
Epoch 57/150
10000/10000 [==============================] - 6s 637us/st

10000/10000 [==============================] - 7s 652us/step - loss: 0.2479 - accuracy: 0.8943 - val_loss: 0.4400 - val_accuracy: 0.8440
Epoch 106/150
10000/10000 [==============================] - 6s 620us/step - loss: 0.2510 - accuracy: 0.8914 - val_loss: 0.4198 - val_accuracy: 0.8400
Epoch 107/150
10000/10000 [==============================] - 6s 629us/step - loss: 0.2443 - accuracy: 0.8964 - val_loss: 0.4505 - val_accuracy: 0.8310
Epoch 108/150
10000/10000 [==============================] - 7s 673us/step - loss: 0.2523 - accuracy: 0.8939 - val_loss: 0.4792 - val_accuracy: 0.8300
Epoch 109/150
10000/10000 [==============================] - 6s 640us/step - loss: 0.2433 - accuracy: 0.8970 - val_loss: 0.4386 - val_accuracy: 0.8390
Epoch 110/150
10000/10000 [==============================] - 7s 654us/step - loss: 0.2399 - accuracy: 0.8978 - val_loss: 0.4624 - val_accuracy: 0.8400
Epoch 111/150
10000/10000 [==============================] - 6s 643us/step - loss: 0.2383 - accuracy: 0.8972

In [37]:
model.save('chat_bot.h5')

In [38]:
predictions = model.predict([inputs_test,queries_test])

In [39]:
predictions[0]

array([2.7715994e-15, 3.1150994e-15, 3.8306002e-15, 2.9551955e-15,
       6.5966015e-04, 3.3855257e-15, 4.3463382e-15, 3.1996706e-15,
       3.7792916e-15, 3.5037365e-15, 3.5202151e-15, 4.0005536e-15,
       3.5409289e-15, 4.0778703e-15, 3.5591842e-15, 3.7787580e-15,
       9.9934036e-01, 3.1427309e-15, 3.6214305e-15, 4.0859052e-15,
       3.4419504e-15, 3.8183305e-15, 3.5551404e-15, 2.9939339e-15,
       3.5741504e-15, 2.9770900e-15, 4.0364855e-15, 3.1923189e-15,
       3.4107122e-15, 3.5023336e-15, 4.3814295e-15, 3.3925328e-15,
       3.1050032e-15, 2.9994781e-15, 2.7211145e-15, 3.8822646e-15,
       2.9281741e-15, 3.2058772e-15], dtype=float32)

In [40]:
test_data[0]

(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'kitchen', '?'],
 'no')

In [41]:
tokenizer.word_index['no']

16

In [42]:
tokenizer.word_index['yes']

4

In [46]:
val_max = np.argmax(predictions[0])
for key,val in tokenizer.word_index.items():
    if val==val_max:
        k = key
print(k)

no


In [47]:
pred = []
for prediction in predictions:
    val_max = np.argmax(prediction)
    if val_max == 16:
        pred.append(0)
    else:
        pred.append(1)

In [51]:
real = []
for data in test_data:
    if data[2] == 'no':
        real.append(0)
    else:
        real.append(1)

In [56]:
from sklearn.metrics import classification_report, accuracy_score

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
print(classification_report(real,pred))

              precision    recall  f1-score   support

           0       0.91      0.73      0.81       503
           1       0.77      0.93      0.84       497

   micro avg       0.83      0.83      0.83      1000
   macro avg       0.84      0.83      0.83      1000
weighted avg       0.84      0.83      0.83      1000



In [59]:
print(confusion_matrix(real,pred))

[[369 134]
 [ 37 460]]


In [60]:
print(accuracy_score(real,pred))

0.829


In [61]:
from pickle import dump

In [62]:
dump(tokenizer,open('chat_bot','wb'))